## _*Quantum Emoticon*_ 

The latest version of this notebook is available on https://github.com/QISKit/qiskit-tutorial.

For more information about how to use the IBM Q experience (QX), consult the [tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0), or check out the [community](https://quantumexperience.ng.bluemix.net/qstage/#/community).

***
### Contributors
James R. Wootton, University of Basel; 
Anna Phan, IBM Research

*Rudy Raymond (IBM Research AI) modified the layout so that the result is not being permuted*

This program aims to do some of the same jobs as "Hello World" does for classical programming.

Specifically:
* It is simple;
* It performs and understandable and relatable task;
* It demonstrates simple principles of the programming language;
* It shows how to produce and look at an output.

Anything that can be done with bits can be done with qubits. Simply leave a qubit in its initialized value for the state *0*, or use an operation with the effect of a NOT gate (such as *X* or *Y*) to rotate it to a *1*. Each qubit then becomes a bit, allowing us to implement "Hello, World!" directly on a quantum computer.

In practice, it is not so straightforward. ASCII encoding of "Hello, World!" requires over 100 bits, and therefore over 100 qubits. Current quantum devices are not yet large enough for the job.

However, two ASCII characters require only 16 (qu)bits. Writing an emoticon like ;) can therefore be done using *ibmqx5*.

The string corresponding to ;) is
    
    ;)  =  '0011101100101001'

To prepare this state we can perform an *X* on qubits 0, 3, 5, 8, 9, 11, 12, 13. Here we number the bits from 0 to 15, from right to left.

Of course, just writing a bit string in some qubits isn't very quantum. Instead we can prepare a superposition of two different emoticons. We choose ;) and 8), which correspond to the bit strings

    8)  =  '0011100000101001'
    ;)  =  '0011101100101001'

Note that these strings differ only on bits 8 and 9. It is therefore only these on which the superposition must be prepared. The superposition will be of the '00' of 8), and the '11' of ;) , and so will be a standard [Bell state](https://github.com/QISKit/qiskit-tutorial/blob/master/2_quantum_information/superposition_and_entanglement.ipynb).

We'll now implement this.

In [1]:
from qiskit import QuantumProgram
from qiskit.tools.visualization import plot_histogram

#import Qconfig and set APIToken and API url
import Qconfig

def make_layout(num_qubits):
    # Create a mapping fixing qubit q0 to q0, q1 to q1, etc.
    # Input is number of qubits for the map
    layout = {}
    for i in range(num_qubits):
        layout[('q', i)] = ('q', i)
    return layout


# set up registers and program
qp = QuantumProgram()

qp.set_api(Qconfig.APItoken, Qconfig.config["url"])
qr = qp.create_quantum_register('q', 16)
cr = qp.create_classical_register('c', 16)
qc = qp.create_circuit('smiley_writer', [qr], [cr])

# rightmost eight (qu)bits have ')' = 00101001
qc.x(qr[0])
qc.x(qr[3])
qc.x(qr[5])

# second eight (qu)bits have superposition of
# '8' = 00111000
# ';' = 00111011
# these differ only on the rightmost two bits
qc.h(qr[9]) # create superposition on 9
qc.cx(qr[9],qr[8]) # spread it to 8 with a cnot
qc.x(qr[11])
qc.x(qr[12])
qc.x(qr[13])

qc.barrier(qr)
# measure
for j in range(16):
    qc.measure(qr[j], cr[j])

We'll now run the circuit on the simulator for 128 shots so we don't need to wait in the *ibmqx5* queue and plot the results. 

{1: [0, 2], 2: [3], 3: [4, 14], 5: [4], 6: [5, 7, 11], 7: [10], 8: [7], 9: [8, 10], 11: [10], 12: [5, 11, 13], 13: [4, 14], 15: [0, 2, 14]}


In [2]:
bconfig = qp.get_backend_configuration('ibmqx5')

print(bconfig["coupling_map"])

shots_sim = 1024
results_sim = qp.execute(["smiley_writer"], backend='ibmqx_hpc_qasm_simulator', 
                         shots = shots_sim, wait=5, timeout=300,
                         coupling_map = bconfig["coupling_map"],
                         initial_layout=make_layout(16))
stats_sim = results_sim.get_counts("smiley_writer")

plot_histogram(stats_sim)

The above results in *stats* tell us how many times each bit string was found as a result. To make our emoticon, we convert the bit strings into ASCII characters. We also calculate the fraction of shots for which each result occurred. The most common results are then printed to screen.

This gives us a list of characters and probabilities. But that isn't very fun. For something more visual, we use matplotlib to create an image in which the all the characters included in the result are printed on top of each other. The alpha channel of each character is set to the fraction of shots for which that result occurred. For our simulator results, this is approximately equally split between 8) and ;).

We will write this transformation and visualisation from results to ASCII characters as a function so we can reuse it later when we run the smiley circuit on the *ibmqx5* device. 

0.4892578125	;)
0.5107421875	8)


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='monospace')

def plot_smiley (stats, shots):
    for bitString in stats:
        char = chr(int( bitString[0:8] ,2)) # get string of the leftmost 8 bits and convert to an ASCII character
        char += chr(int( bitString[8:16] ,2)) # do the same for string of rightmost 8 bits, and add it to the previous character
        prob = stats[bitString] / shots # fraction of shots for which this result occurred
        # create plot with all characters on top of each other with alpha given by how often it turned up in the output
        plt.annotate( char, (0.5,0.5), va="center", ha="center", color = (0,0,0, prob ), size = 300)
        if (prob>0.05): # list prob and char for the dominant results (occurred for more than 5% of shots)
            print(str(prob)+"\t"+char)
    plt.axis('off')
    plt.show()
    
plot_smiley(stats_sim, shots_sim)

We'll now run the smiley circuit on *ibmqx5*, after checking that the device is online. Given the noise on the device, our smiley won't be as clean as our previous run on the simulator.

0.19921875	8)
0.1396484375	;)


In [4]:
#import Qconfig and set APIToken and API url
import Qconfig
qp.set_api(Qconfig.APItoken, Qconfig.config["url"])

if qp.get_backend_status('ibmqx5')['available'] == True:
    shots_device = 1024
    results_device = qp.execute(["smiley_writer"], backend='ibmqx5', shots=shots_device, wait=5, 
                                timeout=300, initial_layout=make_layout(16))
    stats_device = results_device.get_counts("smiley_writer")
    plot_smiley(stats_device, shots_device)

And there you have it. A program to take one of the world's most advanced devices and made a smiley with it. A quantum smiley!

An alternative introduction to this program can be found [here](https://medium.com/@decodoku/making-a-quantum-computer-smile-cee86a6fc1de).

In [ ]:
%run "../version.ipynb"